In [1]:
import pandas as pd
import os

pd.set_option('display.max_columns', 500)

data_path = os.getcwd()[:-len('notebooks')] + 'Data/'
db_credit_card_fraud_detection_small = data_path + 'Credit_card_fraud_detection_small/'
db_beijing_path = data_path + 'Beijing_winter_Olympic_Games/'
transactional_data = data_path + 'Transactional_data/'

In [2]:
os.listdir(transactional_data)

['.DS_Store',
 'goat of tennis.csv',
 'Highest Holywood Grossing Movies.csv',
 'trains vs planes.csv',
 'Loan_Default.csv',
 'New York Citibike Trips.csv',
 'Business Sales Transaction.csv']

# Beijing

In [3]:
os.listdir(db_beijing_path)

['hockey_results.csv',
 'curling_results.csv',
 'athletes.csv',
 'technical_officials.csv',
 'medals_total.csv',
 'coaches.csv',
 'entries_discipline.csv',
 'hockey_players_stats.csv',
 'medals.csv',
 'events.csv']

# Ejercicio Práctica sábado:

Como comité olímpico, nos solicitan analizar la relación que existe entre el número de participantes enviados por cada país y el número de medallas obtenidos en los tres niveles. 

Nos interesa analizar este proceso a nivel país, y nos gustaría poder revisar la cantidad de atletas enviados por cada país (cantidad total, cantidad de mujeres y cantidad de hombres), total de medallas obtenidas, total de medallas de oro, total de medallas de plata y total de medallas de bronce (segmentados por hombres y mujeres). Además de esto, nos interesa contar la cantidad de coaches que envió cada país, de igual forma, divido por mujeres y hombres. Y por último agregar las columnas para los referies, la intención es analizar si la cantidad de personas enviadas por cada país está relacionada con la cantidad de medallas traidas por cada país. 

# Ecommerce

In [4]:
sales_ecommerce = pd.read_csv(transactional_data + 'Business Sales Transaction.csv', low_memory = False)
sales_ecommerce.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,536365,2018-12-01,85123A,Cream Hanging Heart T-Light Holder,1.88,6,17850.0,United Kingdom
1,536365,2018-12-01,71053,White Moroccan Metal Lantern,2.01,6,17850.0,United Kingdom
2,536365,2018-12-01,84406B,Cream Cupid Hearts Coat Hanger,1.91,8,17850.0,United Kingdom
3,536365,2018-12-01,84029G,Knitted Union Flag Hot Water Bottle,2.01,6,17850.0,United Kingdom
4,536365,2018-12-01,84029E,Red Woolly Hottie White Heart,2.01,6,17850.0,United Kingdom


# Ejercicio:

Nos solicitan revisar si es posible establecer una relación entre el país y la demanda de ciertos productos. La idea es que conociendo el país se pueda predecir las cantidades de los top tres productos para cada país (su precio, la cantidad de unidades vendidas) esto agrupado por mes. 

* 1. Unidad muestral: País.
* 2. Quantity variable auxiliar para el cálculo de la demanda.
* 3. Segunda variable auxiliar precio.
* 4. Esto para los 3 productos más vendidos por cada país. 
* 5. Agrupación a nivel mes.

In [5]:
df = sales_ecommerce.copy()
df['key'] = df['Date'].map(lambda x:x.strip()[:7]) # Ejemplo de transformación
df.head(3)

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,key
0,536365,2018-12-01,85123A,Cream Hanging Heart T-Light Holder,1.88,6,17850.0,United Kingdom,2018-12
1,536365,2018-12-01,71053,White Moroccan Metal Lantern,2.01,6,17850.0,United Kingdom,2018-12
2,536365,2018-12-01,84406B,Cream Cupid Hearts Coat Hanger,1.91,8,17850.0,United Kingdom,2018-12


In [6]:
# Proceso de creación de característica
dffinal = pd.DataFrame(index = df[['Country', 'key']].groupby('Country').count().index)

In [7]:
# Filtrado de características
aux = (df[['Country', 'key', 'ProductNo', 'Quantity', 'Price']] # En esta línea está el filtrado
       .groupby(['Country', 'key', 'ProductNo']).agg({'Quantity': 'sum', 'Price': 'mean'}))
aux

Quantity  Price
Country     key     ProductNo                 
Australia   2018-12 20685             4   2.69
                    20725            10   1.75
                    21217            -1   2.99
                    21622             8   2.24
                    21791            12   1.69
...                                 ...    ...
Unspecified 2019-09 84378             2   1.72
                    84991             2   1.58
                    84997C            2   2.12
                    85019B            2   1.69
                    85227            10   1.63

[70902 rows x 2 columns]

In [24]:
# Proceso de extracción
keys = ['2018-12'] + [f'2019-0{i}' for i in range(1,10,1)] + [f'2019-{i}' for i in range(10,13,1)]
rows = []
for f in dffinal.index:
    l = []
    for k in keys:
        try:
            aux_ = dict(aux.loc[(f, k)].sort_values(by=('Quantity'), ascending = False).head(3)['Quantity'])
            aux_2 = dict(aux.loc[(f, k)].sort_values(by=('Quantity'), ascending = False).head(3)['Price'])
            l.append([(i, aux_.get(i), aux_2.get(i)) for i in aux_])
        except:
            l.append([(0,0,0) for i in range(3)])
    rows.append(l)

dff = pd.DataFrame(rows, index = dffinal.index, columns = keys)
dff

,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
Country,,,,,,,,,,,,,
Australia,"[(22915, 120, 1.55), (79067, 50, 1.94), (22196...","[(22492, 576, 1.58), (21915, 252, 1.6749999999...","[(22969, 480, 1.69), (20973, 384, 1.58), (2296...","[(22615, 432, 1.54), (21984, 432, 1.54), (2198...","[(20725, 30, 1.75), (22662, 30, 1.75), (22384,...","[(15036, 600, 1.61), (21902, 576, 1.58), (2190...","[(22492, 576, 1.58), (16161P, 400, 1.55), (227...","[(23295, 408, 1.6175000000000002), (23293, 408...","[(21731, 720, 1.72), (22492, 576, 1.58), (2294...","[(22492, 1152, 1.58), (21915, 732, 1.674999999...","[(22722, 360, 2.02), (23507, 250, 1.55), (2351...","[(23084, 1632, 1.77), (23247, 216, 1.93), (232...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]"
Austria,"[(22153, -48, 1.56)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(21918, 288, 1.55), (22546, 240, 1.55), (2258...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(22492, 36, 1.6), (22693, 24, 1.69), (21723, ...","[(22697, -1, 1.94), (22357, -5, 2.14)]","[(21500, 25, 1.56), (23545, 25, 1.56), (21498,...","[(21402, 24, 1.52), (21403, 24, 1.52), (22814,...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(72817, 24, 1.62), (21788, 24, 1.63), (21789,...","[(23084, 279, 2.12), (22154, 49, 1.59), (23353...","[(22646, 36, 1.56), (15056N, 24, 2.39), (20679..."
Bahrain,"[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(23076, 96, 1.69), (23077, 60, 1.69), (22693,...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]"
Belgium,"[(21212, 148, 1.61), (22417, 144, 1.57), (2197...","[(22489, 144, 1.55), (22540, 48, 1.56), (22544...","[(22740, 48, 1.63), (22540, 48, 1.56), (22531,...","[(20724, 110, 1.62), (22355, 100, 1.61), (2066...","[(21212, 72, 1.58), (21977, 72, 1.58), (21829,...","[(84991, 96, 1.58), (22951, 96, 1.58), (21976,...","[(22492, 108, 1.6), (22740, 96, 1.63), (20724,...","[(84536B, 288, 1.5550000000000002), (22027, 72...","[(23546, 50, 1.56), (23545, 50, 1.56), (21981,...","[(23310, 180, 1.55), (22326, 90, 1.93), (22328...","[(22951, 96, 1.58), (21212, 96, 1.58), (22417,...","[(22629, 112, 1.782), (23084, 108, 1.79), (223...","[(21499, 50, 1.56), (79190B, 27, 1.595), (1615..."
Brazil,"[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(22630, 24, 1.79), (22697, 24, 1.88), (22993,...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]"
Canada,"[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(84755, 16, 1.6), (20886, 12, 1.79), (71459, ...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(37370, 504, 1.66)]","[(84077, 288, 1.53), (20727, 50, 1.75), (10133...","[(47593B, 48, 1.56), (22098, 36, 1.56), (22099...","[(23293, 16, 1.62), (21993, 12, 1.69), (23294,...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]"
Channel Islands,"[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(16161P, 25, 1.56), (21498, 25, 1.56), (21500...","[(21785, 407, 1.6), (22720, 96, 2.14), (22745,...","[(22740, 96, 1.63), (22741, 48, 1.63), (22151,...","[(22908, 24, 1.63), (23206, 20, 1.75), (22666,...","[(21499, 25, 1.56), (21500, 25, 1.56), (21498,...","[(22985, 25, 1.56), (21212, 24, 1.58), (21977,...","[(0, 0, 0), (0, 0, 0), (0, 0, 0)]","[(85099B, 210, 1.79), (85099C,